## 要望
1. Nodeのupload/ariyoshi配下にあるcsvファイルを全て読取り、CSV及びEPC毎のRSSIの基本統計量を出す。  
2. 各CSVで、RSSIの統計量を知りたいEPCのルールは、下記で決まる。  
**0x000000000000000000006 + ファイル名の最初の数値(1桁のものは、0始まりの2桁にする) + 1**  
e.g.)0x000000000000000000006 + TH_10_ALNの数値 + 1 => 0x000000000000000000006101
っていうEPCのRSSIが欲しい 

---

In [1]:
# 全体で使用するライブラリの読み込み と 設定
import pandas as pd # Pandas(https://pandas.pydata.org)をpdとして使用出来る用に
pd.options.display.max_rows=3 # DataFrameの最大表示数(rows)

### ダウンロードしたファイル一覧の取り込み
サーバからファイルを得る為のAPI(Node側)に不具合があり、uploads配下を正常に取得出来ない為、手動で任意のディレクトリにDownload済みであることとする。  
ここでは、**カレントディレクトリ/download/ariyoshi**にダウンロードした想定で進める。

In [2]:
filepaths = [] # フルパスでファイル一覧を格納するリスト

import os
import glob
source_dir = '.\\download\\ariyoshi\\tagonly' # Downloadしたディレクトリ
for dir in os.listdir(source_dir): # source_dir配下のディレクトリ一覧を順次取得してforループを回す(http://docs.python.jp/3.5/library/os.html#os.listdir)
    filepaths.extend(glob.glob(source_dir + '\\' + dir + '\\*.csv'))
len(filepaths) # 確認の為、ファイル数を表示

18

### CSVファイルの取り込み
CSVのファイルを全て読み込み、DataFrameオブジェクトに変換してリストとして保持する。

In [3]:
import re # 正規表現用のライブラリ(https://docs.python.jp/3/library/re.html)
def get_target_epc(filename):
    """
    ファイル名より、RSSIの基本統計量を求めたいEPCを特定し、返却する.
    Args:
        filename: CSVファイル名. TH_数値_で始まる文字列を想定している.
    Returns:
        当該CSVファイルにおいて、RSSIの基本統計量を求めたいEPC.
    """
    target_epc = re.search(r'TH_[0-9]*', filename).group()[3:]
    if len(target_epc) == 1:
        target_epc = "0" + target_epc
    return '0x000000000000000000006' + target_epc + '1'

csvs = [] # CSVファイルを取り込み、DataFrameオブジェクトとして格納するリスト
for filepath in filepaths: # CSVファイル一覧から、ファイルを1つずつ取り出し、変数filepathに入れて、forループを実行
    # https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
    df = pd.read_csv(filepath, skiprows=5, names=('epc', 'time', 'runnum', 'rssi', 'reader', 'channel', 'power', 'antenna'), encoding ='shift-jis')
    
    # https://docs.python.jp/3/library/stdtypes.html#str.rsplit
    file_name = filepath.split('\\')[-1] # ファイルのフルパスを「\\」で分けたリストの最後の要素を取り出す
    df['csv'] =file_name # csvカラムを追加し、ファイル名を値として設定
    df['target_epc'] = get_target_epc(file_name) # target_epcカラムを追加し、基本統計量を求めたいEPCを値として設定
    csvs.append(df)
len(csvs) # 確認の為、オブジェクト化したCSV数を表示

18

In [4]:
# 確認の為、ファイルの中身を1ファイルだけ表示
csvs[0]

,epc,time,runnum,rssi,reader,channel,power,antenna,csv,target_epc
0,0x000000000000000000006101,1.516242e+09,0,-24,0,1,24.0,1,TH_10_ALN-9762-FRA.csv,0x000000000000000000006101
...,...,...,...,...,...,...,...,...,...,...
418,0x108030249016740038843524,1.516242e+09,0,-33,0,1,24.0,4,TH_10_ALN-9762-FRA.csv,0x000000000000000000006101


In [5]:
# CSVファイルのDataFrameを、1ファイルにマージ
merged = pd.concat(csvs)
merged # 確認の為、中身と件数を表示

,epc,time,runnum,rssi,reader,channel,power,antenna,csv,target_epc
0,0x000000000000000000006101,1.516242e+09,0,-24,0,1,24.0,1,TH_10_ALN-9762-FRA.csv,0x000000000000000000006101
...,...,...,...,...,...,...,...,...,...,...
435,0x108040249013510116833645,1.516242e+09,0,-32,0,1,24.0,3,TH_9_ZEPHYR-2.csv,0x000000000000000000006091


In [6]:
# 念の為ファイル数チェック(これがmergedのrow数と一致するはず)
count = 0
for csv in csvs:
    count += len(csv.index)
count

7765

In [7]:
# 基本統計量の算出
# 基本統計量を算出したいEPCのみに絞り込んだ後、csvとepcでグルーピングし、RSSIの基本統計量を求めている
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html
described = merged[merged.epc == merged.target_epc].groupby(['csv', 'epc']).rssi.describe()
described

,,count,mean,std,min,25%,50%,75%,max
csv,epc,,,,,,,,
TH_10_ALN-9762-FRA.csv,0x000000000000000000006101,157.0,-23.203822,2.807359,-29.0,-24.0,-23.0,-21.0,-20.0
...,...,...,...,...,...,...,...,...,...
TH_9_ZEPHYR-2.csv,0x000000000000000000006091,149.0,-23.335570,3.150742,-30.0,-24.0,-24.0,-20.0,-19.0


In [8]:
output_file = ".\\output\\output.csv" # 最終結果出力先

# 基本統計量のDataFrameをCSVのファイルとして出力
# http://pandas.pydata.org/pandas-docs/version/0.20.2/generated/pandas.DataFrame.to_csv.html
described.to_csv(path_or_buf=output_file, encoding='utf-8')